## HW Supervised Learning

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
df = pd.read_excel('dataset/youtube_statistics.xlsx')
df.sample(3)

,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,video_error_or_removed,description,No_tags,desc_len,len_title,publish_date
2988,2017-11-29,Junior NTR MIND BLOWING Reaction On Agnyaatava...,Film Mantra,22,14:42:26,Junior NTR MIND BLOWING Reaction On Agnyaathav...,35181,1206,37,17,False,False,False,Junior NTR MIND BLOWING Reaction On Agnyaathav...,16,546,97,2017-11-27
12638,2018-01-23,SBI Clerk Prelims 2018 | What To Do In 20 Minu...,Mahendra Guru : Online Videos For Govt. Exams,27,05:50:15,"Tseries|""DrVivekBindra:MotivationalSpeaker""|""S...",64663,5035,80,410,False,False,False,FOR PDF FORMAT CLICK HERE :\nhttp://www.mahend...,28,2130,81,2018-01-22
14628,2018-02-03,ਫੇਸਬੁਕ 'ਤੇ ਲਾਈਵ ਖੁਦਕੁਸ਼ੀ ਦੀ ਕੋਸ਼ਿਸ਼,PTC News,25,17:39:50,"punjabi movies|""punjabi politics""|""himachal ne...",21680,231,16,0,True,False,False,http://www.ptcnews.tv/\nhttp://www.facebook.co...,10,173,32,2018-02-02


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36791 entries, 0 to 36790
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   trending_date           36791 non-null  object        
 1   title                   36791 non-null  object        
 2   channel_title           36791 non-null  object        
 3   category_id             36791 non-null  int64         
 4   publish_time            36791 non-null  object        
 5   tags                    36791 non-null  object        
 6   views                   36791 non-null  int64         
 7   likes                   36791 non-null  int64         
 8   dislikes                36791 non-null  int64         
 9   comment_count           36791 non-null  int64         
 10  comments_disabled       36791 non-null  bool          
 11  ratings_disabled        36791 non-null  bool          
 12  video_error_or_removed  36791 non-null  bool  

In [11]:
numeric_col= df.select_dtypes(include=['int64']).columns
object_col = df.select_dtypes(include=['object']).columns
bool_col = df.select_dtypes(include=['bool']).columns
date_time_col = df.select_dtypes(include=['datetime64[ns]']).columns

In [14]:
df[numeric_col].describe().map('{:.2f}'.format)

,category_id,views,likes,dislikes,comment_count,No_tags,desc_len,len_title
count,36791.00,36791.00,36791.00,36791.00,36791.00,36791.00,36791.00,36791.00
mean,21.55,1071490.26,27450.69,1685.36,2714.02,18.94,923.08,70.61
std,6.59,3207149.05,97831.29,16197.32,14978.11,9.84,815.04,22.41
min,1.00,4024.00,0.00,0.00,0.00,1.00,3.00,5.00
25%,23.00,125604.00,879.00,109.00,83.00,12.00,368.00,53.00
50%,24.00,307836.00,3126.00,331.00,336.00,19.00,677.00,74.00
75%,24.00,806631.50,14095.00,1032.00,1314.50,25.00,1237.00,91.00
max,43.00,125432237.00,2912710.00,1545017.00,827755.00,72.00,5136.00,100.00
